# Task 2: Develop a “Tell-the-time” network.

## 1

The problem of correctly telling the time can be formulated either as a multi-class classification
problem (for example, with 12x60=720 classes representing each minute label) or a regression problem
(for example, predicting the number of minutes after 12 o’clock). Therefore, your goal is to come up
with different representations for the labels of your data adapt the output layer of your neural network
and see how it impacts the training time and performance. No matter which architecture and loss
function you will use when reporting results also provide “common sense” accuracy: the absolute value
of the time difference between the predicted and the actual time (e.g., the “common sense” difference
between “predicted” 11:55 and the “target” 0:05 is just 10 minutes and not 11 hours and 50 minutes!).
Minimizing this “common sense” error measure is the main objective of this assignment! Notice that
it is a common situation in Machine Learning: we often train models using one error measure (e.g.,
cross-entropy loss) while the actual performance measure that we are interested in is different, e.g., the
accuracy (the percentage of correctly classified cases).

The dataset can be downloaded from here and it consists of 18000 grayscale images (18000x150x150
or 18000x75x75) contained in ‘images.npy’. The labels for each sample are represented by two integers
(18000x2, ‘labels.npy’ file), that correspond to the hour and minute displayed by the clock. You can see
that each image is rendered from a different angle and rotation and they might contain light reflections from
within the scene making this a non-trivial problem. For your experiments, we suggest splitting your data
into 80/10/10% splits for training/validation and test sets respectively. Remember to shuffle your dataset
as the sample files are ordered. We suggest using the smaller dataset for your initial tests and runs (75x75
images) and then reporting your results on the larger (150x150) datase

### (a) Classification

Treat this as a n-class classification problem. We suggest starting out with a
smaller number of categories e.g. grouping all the samples that are between [3 : 00 − 3 : 30] into
a single category (results in 24 categories in total), and trying to train a CNN model. Once you
have found a working architecture, increase the number of categories by using smaller intervals
for grouping samples to increase the ’common sense accuracy’. Can you train a network using
all 720 different labels? What problems does such a label representation have?

In [1]:

# from tensorflow import keras
import os
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
import keras


2025-10-22 16:17:41.183088: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-22 16:17:41.233931: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-22 16:17:42.322240: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
data_folder = "A1_data_75"
images_path = os.path.join(data_folder, "images.npy")
images = np.load(images_path)
labels_path = os.path.join(data_folder, "labels.npy")
labels = np.load(labels_path)


For your experiments, we suggest splitting your data
into 80/10/10% splits for training/validation and test sets respectively. Remember to shuffle your dataset
as the sample files are ordered. We suggest using the smaller dataset for your initial tests and runs (75x75
images) and then reporting your results on the larger (150x150) dataset

The problem of correctly telling the time can be formulated either as a multi-class classification
problem (for example, with 12x60=720 classes representing each minute label) or a regression problem
(for example, predicting the number of minutes after 12 o’clock). Therefore, your goal is to come up
with different representations for the labels of your data adapt the output layer of your neural network
and see how it impacts the training time and performance. No matter which architecture and loss
function you will use when reporting results also provide “common sense” accuracy: the absolute value
of the time difference between the predicted and the actual time (e.g., the “common sense” difference
between “predicted” 11:55 and the “target” 0:05 is just 10 minutes and not 11 hours and 50 minutes!).
Minimizing this “common sense” error measure is the main objective of this assignment! Notice that
it is a common situation in Machine Learning: we often train models using one error measure (e.g.,
cross-entropy loss) while the actual performance measure that we are interested in is different, e.g., the
accuracy (the percentage of correctly classified cases). Here are some ideas that you should experiment
with when building your models:
2


(a) Classification - treat this as a n-class classification problem. We suggest starting out with a
smaller number of categories e.g. grouping all the samples that are between [3 : 00 −3 : 30] into
a single category (results in 24 categories in total), and trying to train a CNN model. Once you
have found a working architecture, increase the number of categories by using smaller intervals
for grouping samples to increase the ’common sense accuracy’. Can you train a network using
all 720 different labels? What problems does such a label representation have?

In [3]:
#  start with just using the hour labels as classification labels
# extract the hour (first column) from the labels array
print(labels)
def get_cat_labels(labels):
    new_labels = []
    for label in labels:
        label = label[0]* 2 + int(label[1] >= 30)
        new_labels.append(label)
    return np.array(new_labels)
labels = get_cat_labels(labels)
print(labels)


[[ 0  0]
 [ 0  0]
 [ 0  0]
 ...
 [11 59]
 [11 59]
 [11 59]]
[ 0  0  0 ... 23 23 23]


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"TensorFlow detected {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"  - {gpu.name}")
else:
    print("TensorFlow did NOT detect any GPUs. It will use the CPU.")

TensorFlow detected 1 GPU(s):
  - /physical_device:GPU:0


In [5]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_valid shape:", X_valid.shape)
print("y_valid shape:", y_valid.shape)
print("X_test shape:", X_test.shape)
printl = labels.reshape(-1, 1)
print("y_test shape:", y_test.shape)


X_train shape: (14400, 75, 75)
y_train shape: (14400,)
X_valid shape: (1800, 75, 75)
y_valid shape: (1800,)
X_test shape: (1800, 75, 75)
y_test shape: (1800,)


In [6]:
%pip list

Package                 Version
----------------------- -----------
absl-py                 2.3.1
asttokens               3.0.0
astunparse              1.6.3
Bottleneck              1.4.2
brotlicffi              1.0.9.2
certifi                 2025.10.5
cffi                    2.0.0
charset-normalizer      3.4.4
comm                    0.2.3
debugpy                 1.8.16
decorator               5.2.1
exceptiongroup          1.3.0
executing               2.2.1
filelock                3.17.0
flatbuffers             25.9.23
fsspec                  2025.7.0
gast                    0.6.0
gmpy2                   2.2.1
google-pasta            0.2.0
grpcio                  1.76.0
h5py                    3.15.1
idna                    3.11
importlib_metadata      8.7.0
ipykernel               7.0.1
ipython                 9.6.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
Jinja2                  3.1.6
joblib                  1.5.2
jupyter_client          8.6.3
jupyter_core     

In [7]:
def common_sense_loss(y_true, y_pred):
    """
    """
    y_pred_class = tf.argmax(y_pred, axis=1)
    y_true_float = tf.cast(tf.squeeze(y_true), dtype=tf.float32)
    y_pred_float = tf.cast(y_pred_class, dtype=tf.float32)
    diff = tf.abs(y_true_float - y_pred_float)
    cyclical_diff = tf.minimum(diff, 12.0 - diff)
    # Return the mean of these cyclical differences for the batch
    return tf.reduce_mean(cyclical_diff)


In [8]:
max_pool = keras.layers.MaxPool2D(pool_size=2)
# avg_pool = keras.layers.AveragePooling2D(pool_size=2)
model = keras.models.Sequential([
    keras.Input(shape=(75, 75, 1)),
    # Block 1
    keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)), # Output shape: (37, 37, 32)

    # Block 2
    keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"), # <-- Note: 64 filters again
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (18, 18, 64)

    # Block 3
    keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"), # <-- Note: 128 filters again
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (9, 9, 128)

    # Block 4
    keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2), # Output shape: (4, 4, 256)

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="leaky_relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="leaky_relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(24, activation="softmax")
])
model.compile(loss='sparse_categorical_crossentropy',
optimizer=keras.optimizers.Adam(learning_rate=0.001),
metrics=[common_sense_loss,"Accuracy"
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    # callbacks=[early_stopping]
)
#evaluate the model on the test set
test_loss,test_csl, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
#base:0.8420000076293945
#leaky: 0.8525000214576721
#leaky + L2regularization: 0.8472999930381775
#leaky + batch normalization: 0.8978000283241272

(print(tf.__version__))
# metrics=["accuracy"])

I0000 00:00:1761142665.995530    7134 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6119 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-10-22 16:17:46.333888: W external/local_xla/xla/service/gpu/llvm_gpu_backend/default/nvptx_libdevice_path.cc:41] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykernel_launcher.runfiles/cuda_nvdisasm
  ipykernel_launcher.runfiles/nvidia_nvshmem
  ipykern/cuda_nvcc
  ipykern/cuda_nvdisasm
  ipykern/nvidia_nvshmem
  
  /usr/local/cuda
  /opt/cuda
  /home/ollie/anaconda3/envs/idl-fix/lib/python3.12/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/ollie/anaconda3/envs/idl-fix/lib/python3.12

Epoch 1/30


2025-10-22 16:17:50.029328: I external/local_xla/xla/service/service.cc:163] XLA service 0x7eeba00032c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-22 16:17:50.029376: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-10-22 16:17:50.132618: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-22 16:17:50.672520: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91400
2025-10-22 16:17:50.899013: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-22 16:17:50.

InternalError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/ollie/anaconda3/envs/idl-fix/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/ollie/anaconda3/envs/idl-fix/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ollie/anaconda3/envs/idl-fix/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 758, in start

  File "/home/ollie/anaconda3/envs/idl-fix/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/home/ollie/anaconda3/envs/idl-fix/lib/python3.12/asyncio/base_events.py", line 645, in run_forever

  File "/home/ollie/anaconda3/envs/idl-fix/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once

  File "/home/ollie/anaconda3/envs/idl-fix/lib/python3.12/asyncio/

In [ ]:
print(model.evaluate(X_test, y_test))

### (b) Regression

try to build a network that predicts the time using a single output node in the
following format: [”03 : 00” → y = 3.0]; [”05 : 30” → y = 5.5], where categorical labels of hours
and minutes get transformed to a single continuous value. What kind of loss function would you
need to use for such a task? What kind of problems does such a representation have?

### (c) Multi-head models

### (d) Label transformation

## 2
Use the knowledge gained by working with other datasets in the previous parts of this assignment to
optimize your final models and decrease the error of telling the time as much as possible (common
sense error of below 10 minutes is achievable using relatively simple CNN architectures). You should
also compare the different ways of representing your labels and different neural network output layer
combinations. You should use an 80:20% ratio for the train/test sets respectively. Document your
experiments and findings in the report.